<a href="https://colab.research.google.com/github/emielsteegh/mdwnlp/blob/main/Experiments/with-task-adaptive-pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the code to run most experiments
It starts with the task adaption of a lanugae model
And ends with a quick wrapper to run multiple experiments

**IMPORTANT** the experiments use a *modified* version of the (Contrastive Active Learning library)[https://github.com/mourga/contrastive] by Katerina Margatina (github/mourga)

In [ ]:
# pray for an A100 GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU, please use one')
else:
  print(gpu_info)


Sun Jul 17 21:45:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cont Pretraining

In [ ]:
!pip install transformers==4.20 datasets nlp

In [ ]:
BASE = '/Experiments'
TAPT_BERTJE = BASE+'/models/taptBERTje/'

In [ ]:
# imports

from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch

import os
import re
import itertools
import random
import sys

In [ ]:
path = '/Experiments/models'
tapt_model_path = os.path.join(path, "taptBERTje")


In [ ]:
# load model

tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
model = AutoModelForPreTraining.from_pretrained("GroNLP/bert-base-dutch-cased")  # PyTorch

Some weights of BertForPreTraining were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.tokenize("Ondernemingsrecht")

# Data loading

In [ ]:
# load dataset

dataset = load_dataset("Rodekool/ornl") 


Using custom data configuration default
Reusing dataset ornl (/root/.cache/huggingface/datasets/Rodekool___ornl/default/0.0.0/469bdfd028c8f296ef2a644744dd4dd0a573f621b782b3e25e5cc4a3613dca63)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# function to split dataset into sentences 

def text_breaker(t):
  # t = re.sub('\. ','\n', t)
  t = re.sub('( ){1,}|(\\xa0){1,}', ' ', t)
  ts = re.split(' \\n |\\n', t)
  ts = list(filter(lambda l: len(l.split(' ')) >= 5 , ts))
  return ts


In [ ]:
sample_count = 50_000
train_data = random.sample(dataset['train']['text'], sample_count)


In [ ]:
train_data_pars_of_lines = [text_breaker(lines) for lines in train_data]
train_data_lines = list(itertools.chain(*train_data_pars_of_lines))
train_data_sentences = [item for sentence in train_data_lines for item in sentence.split('. ') if item != '']

print(len(train_data_pars_of_lines))
print(len(train_data_lines))
print(len(train_data_sentences))


50000
1015179
1961856


# fixing the tokenizer

In [ ]:
tapt_tokenizer = tokenizer

tapt_tokenizer.vocab_size

30073

In [ ]:
batch_size = 1_000

def batch_iterator():
    for i in range(0, len(train_data_sentences), batch_size):
        yield train_data_sentences[i : i + batch_size]

In [ ]:
tapt_tokenizer = tapt_tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=30073)

In [ ]:
def get_vocab_sim(vocab_a, vocab_b):
  # as per overlap/vocab_size and jaccard similarity index
  a_set = set(vocab_a.keys())
  b_set = set(vocab_b.keys())
  intersection = a_set.intersection(b_set)
  return len(intersection)/max(len(a_set), len(b_set)),  len(intersection)/len(a_set.union(b_set))

In [ ]:
set([1,2,3]).union(set([2,3,4]))

{1, 2, 3, 4}

In [ ]:
get_vocab_sim(tokenizer.vocab, tapt_tokenizer.vocab)

(0.37964286901872113, 0.23429579921607258)

# NSP

In [ ]:
sentence_a = []
sentence_b = []
label = []

for paragraph in train_data_lines:
    sentences = [
        sentence for sentence in paragraph.split('. ') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        # if ((sentences[int(start)].startswith(('ECLI', ' ECLI')))
        #  or (sentences[start+1].startswith(('ECLI', ' ECLI')))):
        #   start = random.randint(0, num_sentences-2)
        #   if ((sentences[int(start)].startswith(('ECLI', ' ECLI')))
        #   or (sentences[start+1].startswith(('ECLI', ' ECLI')))):
        #     start = random.randint(0, num_sentences-2)

        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, len(train_data_sentences)-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(train_data_sentences[index])
            label.append(1)


In [ ]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')
    

1
Vernietiging besluit
---
De rechtbank Amsterdam heeft in haar uitspraak van 21 juli 2008 het beroep tegen het besluit van 10 april 2007 ongegrond verklaard

0
M.C.P
---
van den Berg te Someren (belanghebbende)

1
J.T.J.A
---
Voor het procesverloop en de beslissing in eerste aanleg verwijst het hof naar de bestreden beschikking.



In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt',
                   max_length=512, truncation=True, padding='max_length')

In [ ]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T

# MLM

In [ ]:
test = tokenizer('ik ben een [MASK]', return_tensors='pt',
    max_length=10,
    truncation=True,
    padding='max_length')

test.input_ids


tensor([[    1, 13604,  9092, 11130,     4,     2,     3,     3,     3,     3]])

In [ ]:
# copy label input ids to be the labels

inputs['labels'] = inputs.input_ids.detach().clone()


In [ ]:
# create a masking layer

rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 1) * (inputs.input_ids != 2) * (inputs.input_ids != 3)
 # 1,2,3 are special tokens

for row in range(mask_arr.shape[0]):
  selection = torch.flatten(mask_arr[row].nonzero()).tolist()
  inputs.input_ids[row, selection] = 4 # masking token

In [ ]:
class RechtspraakDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)
    

In [ ]:
train_dataset = RechtspraakDataset(inputs)


In [ ]:
dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = 16,
    shuffle = True)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30073, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
from transformers import AdamW
from tqdm import tqdm

model.train()
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# training

In [ ]:
epochs = 1
losses = []

for epoch in range(epochs):
  loop = tqdm(dataloader, leave=True)
  for batch in loop:
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # pull all tensor batches required for training
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    next_sentence_label = batch['next_sentence_label'].to(device)
    labels = batch['labels'].to(device)
    # process
    outputs = model(input_ids, attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    next_sentence_label=next_sentence_label,
                    labels=labels)
    # extract loss
    loss = outputs.loss
    # calculate loss for every parameter that needs grad update
    loss.backward()
    losses.append(loss.item())
    # update parameters
    optim.step()
    # print relevant info to progress bar
    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())


  0%|          | 0/26721 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 0: 100%|██████████| 26721/26721 [1:13:16<00:00,  6.08it/s, loss=0.267]


In [ ]:
print(tapt_path)

/content/drive/MyDrive/UvA/thesis/models/taptBERTje


In [ ]:
# save model
model.save_pretrained(TAPT_BERTJE)

In [ ]:
# Python program to store list to file using pickle module
import pickle

with open(TAPT_BERTJE+'/losses.pkl', 'wb') as f:
  pickle.dump(losses, f)

In [ ]:
with open(TAPT_BERTJE+'/losses.pkl', 'rb') as f:
  losses_load = pickle.load(f)

In [ ]:
# original_model = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # PyTorch

# tapt_model = AutoModel.from_pretrained(TAPT_BERTJE)
tapt_token = AutoTokenizer.from_pretrained(TAPT_BERTJE)

In [ ]:
original_model.state_dict()

In [ ]:
tapt_model.state_dict()

# AL LOOP WITH NEW MODEL

In [ ]:
BASE = '/Experiments'
CAL = '/contrastive-active-learning' # '/contrastive-active-learning'
REQS = BASE+CAL+'/requirements.txt'
TAPT_BERTJE = BASE+'/models/taptBERTje/'
AL_SCRIPT = BASE+CAL+'/run_al.py'

In [ ]:
!pip install -r $REQS

In [ ]:
# import random
# seeds = random.sample(range(0,9999), 5)
# seeds.sort()
seeds = [672, 2451, 5262, 7763, 9105]

## Hyper parameters that stay put

params = {
    
    'dataset_name'              : 'ornl',
    'budget'                    : '8%',         # default 15%
    'per_gpu_train_batch_size'  : 32,
    'max_seq_length'            : 256,
    'resume'                    : False,
    'cap_training_pool'         : 50000,
    'init'                      : 'random',
    'init_train_data'           : '1%',         # default 1%
    'acquisition_size'          : '1%',         # default 2%
    'model_name_or_path'        : 'wietsedv/bert-base-dutch-cased' # bert-base-cased # wietsedv is an older version, thats okay

    # l rate 2e-5
    # adamw opt with epsilon 1e-8
    # eval 5x p epoch
}

def make_flags(params_dict: dict, acq: str, seed: int, dataset: str = 'ornl'):
  #  flags, acquisition function and seed to string of flags for run_al.py
  #  'random' 'cal' 'entropy' 'zip'
  params_dict['acquisition'] = acq
  params_dict['seed'] = seed
  params_dict['dataset_name'] = dataset
  flags_string = ' '.join([f'--{f} {v}' for f, v in params_dict.items()])
  print("\n", flags_string, "\n")
  return flags_string

_ = make_flags(params, acq = 'entropy', seed = seeds[0])


 --dataset_name ornl --budget 8% --per_gpu_train_batch_size 32 --max_seq_length 256 --resume False --cap_training_pool 50000 --init random --init_train_data 1% --acquisition_size 1% --model_name_or_path wietsedv/bert-base-dutch-cased --acquisition entropy --seed 672 



# ORNL20

In [ ]:
for current_seed in seeds[:3]:
  print(current_seed)
  flags = make_flags(params, acq = 'cal', seed = current_seed, dataset = 'ornl20')
  %run $AL_SCRIPT $flags

672

 --dataset_name ornl20 --budget 8% --per_gpu_train_batch_size 32 --max_seq_length 256 --resume False --cap_training_pool 50000 --init random --init_train_data 1% --acquisition_size 1% --model_name_or_path wietsedv/bert-base-dutch-cased --acquisition cal --seed 672 

torch: 1.12.0+cu113
cuda: 11.3
Cuda available: True


07/17/2022 21:48:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


device: cuda:0
output_dir=/content/drive/MyDrive/UvA/thesis/contrastive-active-learning/checkpoints/ornl20_bert_cal_672/ornl20_bert-cls
Created /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/checkpoints/ornl20_bert_cal_672/ornl20_bert-cls



 /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20 





07/17/2022 21:48:52 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_train_ornl20_original
07/17/2022 21:48:59 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_dev_ornl20_original
07/17/2022 21:49:01 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_test_ornl20_original



train set stats: class 21: 17% class 20: 17% class 17: 7% class 25: 17% class 3: 17% class 2: 3% class 23: 5% class 18: 1% class 4: 1% class 1: 5% class 9: 0% class 11: 3% class 14: 0% class 5: 1% class 15: 1% class 13: 0% class 6: 2% class 22: 0% class 10: 0% class 12: 1% class 19: 0% class 0: 0% class 16: 0% class 8: 0% class 7: 0% 
validation set stats: class 25: 17% class 20: 17% class 3: 17% class 21: 17% class 17: 7% class 0: 0% class 6: 2% class 23: 5% class 2: 3% class 1: 5% class 11: 3% class 12: 1% class 4: 1% class 18: 1% class 15: 1% class 14: 0% class 5: 1% class 10: 0% class 9: 0% class 19: 0% class 8: 0% class 22: 1% class 16: 0% class 13: 0% class 7: 0% 
test set stats: class 3: 17% class 5: 1% class 25: 18% class 1: 5% class 15: 1% class 20: 17% class 21: 17% class 2: 3% class 17: 7% class 6: 2% class 4: 1% class 23: 5% class 10: 0% class 0: 0% class 11: 2% class 8: 0% class 12: 1% class 22: 0% class 9: 0% class 16: 0% class 18: 1% class 14: 0% class 13: 0% class 19: 

...


Evaluating robustness! Start testing on OOD test set!


Evaluating Dpool!



07/18/2022 06:09:21 - INFO - utilities.data_loader -   Loading features from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_train_bert-base-dutch-cased_256_ornl20_original
07/18/2022 06:09:39 - INFO - utilities.data_loader -   Selecting subsample...
07/18/2022 06:09:45 - INFO - utilities.trainers -   ***** Running evaluation  *****
07/18/2022 06:09:45 - INFO - utilities.trainers -     Num examples = 46500
07/18/2022 06:09:45 - INFO - utilities.trainers -     Batch size = 256
Evaluating:   0%|          | 0/182 [00:00<?, ?it/s]

MC samples N=None


Evaluating: 100%|██████████| 182/182 [06:11<00:00,  2.04s/it]
07/18/2022 06:15:56 - INFO - utilities.trainers -   ***** Eval results  *****
07/18/2022 06:15:56 - INFO - utilities.trainers -     acc = 0.8680430107526882
07/18/2022 06:15:56 - INFO - utilities.trainers -     acc_and_f1 = 0.5937345925936267
07/18/2022 06:15:56 - INFO - utilities.trainers -     f1 = 0.3194261744345652
07/18/2022 06:15:56 - INFO - utilities.data_loader -   Loading features from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_train_bert-base-dutch-cased_256_ornl20_original
07/18/2022 06:16:16 - INFO - utilities.data_loader -   Selecting subsample...
07/18/2022 06:16:20 - INFO - utilities.trainers -   ***** Running evaluation  *****
07/18/2022 06:16:20 - INFO - utilities.trainers -     Num examples = 3500
07/18/2022 06:16:20 - INFO - utilities.trainers -     Batch size = 256
Evaluating: 100%|██████████| 14/14 [00:27<00:00,  1.99s/it]
07/18/2022 06:16:48 - INFO - uti



************
End of iteration 7:
Train loss 1.4643, Val loss 0.6009200492075512, Test loss 0.6063534968039569
Annotated 500 samples
Current labeled (training) data: 4000 samples
Remaining budget: 0 (in samples)
************

Saving json with the results....
The end!....


In [ ]:
flags = make_flags(params, acq = 'random', seed = 5262, dataset = 'ornl20')
%run $AL_SCRIPT $flags

In [ ]:
params['acquisition_size'] = '98%' 
params['init_train_data'] = '1%'
params['budget'] = '100%'

flags = make_flags(params, acq = 'random', seed = 5262, dataset = 'ornl20')
%run $AL_SCRIPT $flags

07/17/2022 12:18:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False



 --dataset_name ornl20 --budget 100% --per_gpu_train_batch_size 32 --max_seq_length 256 --resume False --cap_training_pool 50000 --init random --init_train_data 1% --acquisition_size 98% --model_name_or_path wietsedv/bert-base-dutch-cased --acquisition random --seed 5262 

device: cuda:0
output_dir=/content/drive/MyDrive/UvA/thesis/contrastive-active-learning/checkpoints/ornl20_bert_random_5262/ornl20_bert-cls



 /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20 





07/17/2022 12:18:33 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_train_ornl20_original
07/17/2022 12:18:35 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_dev_ornl20_original
07/17/2022 12:18:36 - INFO - utilities.data_loader -   Loading dataset from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_test_ornl20_original



train set stats: class 21: 17% class 20: 17% class 17: 7% class 25: 17% class 3: 17% class 2: 3% class 23: 5% class 18: 1% class 4: 1% class 1: 5% class 9: 0% class 11: 3% class 14: 0% class 5: 1% class 15: 1% class 13: 0% class 6: 2% class 22: 0% class 10: 0% class 12: 1% class 19: 0% class 0: 0% class 16: 0% class 8: 0% class 7: 0% 
validation set stats: class 25: 17% class 20: 17% class 3: 17% class 21: 17% class 17: 7% class 0: 0% class 6: 2% class 23: 5% class 2: 3% class 1: 5% class 11: 3% class 12: 1% class 4: 1% class 18: 1% class 15: 1% class 14: 0% class 5: 1% class 10: 0% class 9: 0% class 19: 0% class 8: 0% class 22: 1% class 16: 0% class 13: 0% class 7: 0% 
test set stats: class 3: 17% class 5: 1% class 25: 18% class 1: 5% class 15: 1% class 20: 17% class 21: 17% class 2: 3% class 17: 7% class 6: 2% class 4: 1% class 23: 5% class 10: 0% class 0: 0% class 11: 2% class 8: 0% class 12: 1% class 22: 0% class 9: 0% class 16: 0% class 18: 1% class 14: 0% class 13: 0% class 19: 

07/17/2022 12:19:43 - INFO - utilities.trainers -   Training/evaluation parameters Namespace(acc_best=0, acc_best_iteration=0, acquisition='random', acquisition_size=49000, adam_epsilon=1e-08, bert_rep=False, bert_score=False, binary=False, budget=(100, True), cache_dir='/content/drive/MyDrive/UvA/thesis/contrastive-active-learning/cache', cap_training_pool=50000, ce=False, cls=True, conf_mask=False, conf_thresh=0.0, config_name='', current_output_dir='/content/drive/MyDrive/UvA/thesis/contrastive-active-learning/checkpoints/ornl20_bert_random_5262/ornl20_bert-cls/iter-1', data_dir='/content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20', dataset_name='ornl20', device=device(type='cuda', index=0), do_eval=True, do_lower_case=False, do_train=True, eval_all_checkpoints=False, evaluate_during_training=True, fp16=False, fp16_opt_level='O1', gpu='0', gradient_accumulation_steps=1, indicator=None, init='random', init_train_data=500, knn_lab=False, learning_rate=2e-05, loca

...

{"eval_acc": 0.961699064074874, "eval_f1": 0.7416385472342719, "eval_acc_and_f1": 0.8516688056545729, "eval_loss": 0.14804877447230474, "learning_rate": 0.0, "train_loss": 0.001062721211439101, "step": 4641}


07/17/2022 13:21:31 - INFO - utilities.trainers -   ***** Running evaluation iter-2_trial1 *****
07/17/2022 13:21:31 - INFO - utilities.trainers -     Num examples = 6945
07/17/2022 13:21:31 - INFO - utilities.trainers -     Batch size = 256
Evaluating: 100%|██████████| 28/28 [00:27<00:00,  1.03it/s]
07/17/2022 13:21:58 - INFO - utilities.trainers -   ***** Eval results iter-2_trial1 *****
07/17/2022 13:21:58 - INFO - utilities.trainers -     acc = 0.9614110871130309
07/17/2022 13:21:58 - INFO - utilities.trainers -     acc_and_f1 = 0.832469822079246
07/17/2022 13:21:58 - INFO - utilities.trainers -     f1 = 0.7035285570454611
07/17/2022 13:22:01 - INFO - utilities.data_loader -   Loading features from cached file /content/drive/MyDrive/UvA/thesis/contrastive-active-learning/data/ORNL20/cached_test_bert-base-dutch-cased_256_ornl20_original



Done Training!


Start Testing on test set!



07/17/2022 13:22:07 - INFO - utilities.trainers -   ***** Running evaluation  *****
07/17/2022 13:22:07 - INFO - utilities.trainers -     Num examples = 34722
07/17/2022 13:22:08 - INFO - utilities.trainers -     Batch size = 256
Evaluating: 100%|██████████| 136/136 [02:15<00:00,  1.00it/s]
07/17/2022 13:24:23 - INFO - utilities.trainers -   ***** Eval results  *****
07/17/2022 13:24:23 - INFO - utilities.trainers -     acc = 0.9580957318126836
07/17/2022 13:24:23 - INFO - utilities.trainers -     acc_and_f1 = 0.8038705445329088
07/17/2022 13:24:23 - INFO - utilities.trainers -     f1 = 0.6496453572531341



Evaluating robustness! Start testing on OOD test set!


Evaluating Dpool!



************
End of iteration 2:
Train loss 0.2827, Val loss 0.14353272039443254, Test loss 0.15530538356260343
Annotated 500 samples
Current labeled (training) data: 50000 samples
Remaining budget: 0 (in samples)
************

Saving json with the results....
The end!....


## new bert loop

In [ ]:
for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'cal', seed = current_seed)
  %run $AL_SCRIPT $flags

In [ ]:
params['model_name_or_path'] = TAPT_BERTJE

for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'cal', seed = current_seed)
  %run $AL_SCRIPT $flags

In [ ]:
params['model_name_or_path'] = TAPT_BERTJE

for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'random', seed = current_seed)
  %run $AL_SCRIPT $flags

---
from here after we may again

In [ ]:
seeds = [3, 4, 5]

params['model_name_or_path'] = 'wietsedv/bert-base-dutch-cased'
params['max_seq_length'] = 256

for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'random', seed = current_seed)
  %run $AL_SCRIPT $flags

In [ ]:
seeds = [512001, 512002, 512003, 512004, 512005]
params['model_name_or_path'] = TAPT_BERTJE
params['max_seq_length'] = 512 # <---------------------- this one
params['per_gpu_train_batch_size'] = 16
params['acquisition_size'] = '250' # <------------------ and this 

for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'entropy', seed = current_seed)
  %run $AL_SCRIPT $flags

## full supervision

In [ ]:
params['model_name_or_path'] = TAPT_BERTJE
params['acquisition_size'] = '1%' # <------------------ and this 
params['init_train_data'] = '98%' # <------------------ and this 
params['budget'] = '100%' # <------------------ and this 

for current_seed in seeds:
  print(current_seed)
  flags = make_flags(params, acq = 'random', seed = current_seed)
  %run $AL_SCRIPT $flags